# Wide & Deep 电影推荐系统

本系统包含完整的推荐流程：
1. **数据处理**：加载和预处理 MovieLens 数据
2. **特征工程**：用户特征、电影特征、类型向量化
3. **模型构建**：Wide & Deep 神经网络架构
4. **召回**：多路召回策略（热门、类型、协同过滤）
5. **重排序**：使用 Wide & Deep 模型精排
6. **推荐**：生成个性化推荐结果

## 1. 导入库和加载推荐系统

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 导入推荐系统
from wide_deep_recommender import MovieRecommender

# 设置显示
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']  # 支持中文
plt.rcParams['axes.unicode_minus'] = False

print("导入成功！")

## 2. 初始化推荐系统并加载数据

In [ ]:
# 数据路径
DATA_PATH = './data/ml-10M100K'

# 创建推荐系统
recommender = MovieRecommender(DATA_PATH)

# 准备数据
ratings, movies, train_data, user_stats, movie_features, all_genres = recommender.prepare_data()

## 3. 数据探索分析

In [ ]:
print("="*60)
print("数据概览")
print("="*60)
print(f"用户数量: {ratings['user_id'].nunique():,}")
print(f"电影数量: {movies['movie_id'].nunique():,}")
print(f"评分数量: {len(ratings):,}")
print(f"电影类型: {len(all_genres)}")
print(f"\n电影类型列表: {', '.join(all_genres)}")

print("\n" + "="*60)
print("评分分布")
print("="*60)
print(ratings['rating'].value_counts().sort_index())

In [ ]:
# 可视化评分分布
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 评分分布
axes[0, 0].hist(ratings['rating'], bins=10, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('评分分布', fontsize=14)
axes[0, 0].set_xlabel('评分')
axes[0, 0].set_ylabel('频次')

# 用户评分数量分布
user_rating_counts = ratings.groupby('user_id').size()
axes[0, 1].hist(user_rating_counts, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('用户评分数量分布', fontsize=14)
axes[0, 1].set_xlabel('评分数量')
axes[0, 1].set_ylabel('用户数')
axes[0, 1].set_yscale('log')

# 电影评分数量分布
movie_rating_counts = ratings.groupby('movie_id').size()
axes[1, 0].hist(movie_rating_counts, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].set_title('电影评分数量分布', fontsize=14)
axes[1, 0].set_xlabel('评分数量')
axes[1, 0].set_ylabel('电影数')
axes[1, 0].set_yscale('log')

# 电影类型分布
genre_counts = {}
for genres_str in movies['genres']:
    for genre in genres_str.split('|'):
        genre_counts[genre] = genre_counts.get(genre, 0) + 1

genres = list(genre_counts.keys())
counts = list(genre_counts.values())
axes[1, 1].barh(genres, counts, alpha=0.7)
axes[1, 1].set_title('电影类型分布', fontsize=14)
axes[1, 1].set_xlabel('电影数量')

plt.tight_layout()
plt.show()

## 4. 查看训练数据

In [ ]:
print("训练数据示例:")
print(train_data.head(10))

print("\n训练数据统计:")
print(train_data.describe())

print("\n特征列:")
print(train_data.columns.tolist())

## 5. 构建和训练 Wide & Deep 模型

### Wide & Deep 架构说明：

**Wide 部分**（线性模型）：
- 输入：用户×电影交叉特征 + 用户统计特征 + 电影统计特征 + 类型特征
- 层结构：单层线性变换
- 作用：记忆历史交互模式

**Deep 部分**（深度神经网络）：
- 输入层：用户ID、电影ID、统计特征、类型特征
- 嵌入层：将稀疏ID转换为稠密向量（32维）
- 隐藏层：
  - Layer 1: 256 units + ReLU + Dropout(0.3) + BatchNorm
  - Layer 2: 128 units + ReLU + Dropout(0.3) + BatchNorm
  - Layer 3: 64 units + ReLU + Dropout(0.3) + BatchNorm
- 作用：学习特征的高阶组合，提高泛化能力

**联合训练层**：
- Wide 和 Deep 的输出相加
- Sigmoid 激活函数输出预测概率

In [ ]:
# 构建和训练模型（可调整 epochs 和 batch_size）
# 注意：完整训练可能需要较长时间，建议先用小的 epochs 测试

history = recommender.build_and_train(
    train_data, 
    user_stats, 
    movie_features, 
    all_genres,
    epochs=5,  # 可以调整为 10-20 以获得更好效果
    batch_size=2048
)

## 6. 可视化训练过程

In [ ]:
# 绘制训练曲线
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(history.history['loss'], label='训练损失', marker='o')
axes[0].plot(history.history['val_loss'], label='验证损失', marker='s')
axes[0].set_title('模型损失', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='训练准确率', marker='o')
axes[1].plot(history.history['val_accuracy'], label='验证准确率', marker='s')
axes[1].set_title('模型准确率', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# AUC
axes[2].plot(history.history['auc'], label='训练 AUC', marker='o')
axes[2].plot(history.history['val_auc'], label='验证 AUC', marker='s')
axes[2].set_title('模型 AUC', fontsize=14)
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('AUC')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n最终指标:")
print(f"验证集 Loss: {history.history['val_loss'][-1]:.4f}")
print(f"验证集 Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"验证集 AUC: {history.history['val_auc'][-1]:.4f}")

## 7. 设置召回和重排序引擎

In [ ]:
# 设置推荐引擎
recommender.setup_engines(ratings, movies, movie_features, user_stats, all_genres)
print("推荐引擎设置完成！")

## 8. 生成推荐

### 推荐流程：
1. **召回阶段**：使用多路召回策略获取候选电影（~200部）
   - 热门召回：最受欢迎的电影
   - 类型召回：基于用户喜好的类型
   - 协同过滤：基于相似用户的偏好

2. **重排序阶段**：使用 Wide & Deep 模型对候选电影打分并排序
   - 计算每部电影的预测评分
   - 返回 Top-K 推荐结果

In [ ]:
# 选择一个用户进行推荐
test_user_id = 1

# 查看用户历史
user_history = ratings[ratings['user_id'] == test_user_id].merge(
    movies, on='movie_id'
).sort_values('rating', ascending=False)

print(f"用户 {test_user_id} 的历史评分（Top 10 高分电影）:")
print(user_history[['title', 'genres', 'rating']].head(10))

In [ ]:
# 生成推荐
recommended_movie_ids, scores = recommender.recommend(test_user_id, top_k=10)

# 获取推荐电影信息
recommended_movies = recommender.get_movie_info(recommended_movie_ids, movies)

print(f"\n为用户 {test_user_id} 推荐的电影 (Top 10):")
print("="*80)

for idx, (movie_id, score) in enumerate(zip(recommended_movie_ids, scores), 1):
    movie_info = movies[movies['movie_id'] == movie_id].iloc[0]
    print(f"{idx}. {movie_info['title']}")
    print(f"   类型: {movie_info['genres']}")
    print(f"   预测评分: {score:.4f}")
    print()

## 9. 测试多个用户的推荐效果

In [ ]:
# 测试多个用户
test_users = [1, 100, 500, 1000, 5000]

for user_id in test_users:
    if user_id not in ratings['user_id'].values:
        print(f"用户 {user_id} 不存在，跳过")
        continue
        
    print("\n" + "="*80)
    print(f"用户 {user_id} 的推荐")
    print("="*80)
    
    # 用户历史偏好
    user_hist = ratings[ratings['user_id'] == user_id].merge(
        movies, on='movie_id'
    ).sort_values('rating', ascending=False).head(3)
    
    print("\n用户喜欢的电影:")
    for _, row in user_hist.iterrows():
        print(f"  - {row['title']} ({row['genres']}) - 评分: {row['rating']}")
    
    # 生成推荐
    rec_movies, rec_scores = recommender.recommend(user_id, top_k=5)
    
    print("\n推荐电影:")
    for idx, (movie_id, score) in enumerate(zip(rec_movies, rec_scores), 1):
        movie = movies[movies['movie_id'] == movie_id].iloc[0]
        print(f"  {idx}. {movie['title']} ({movie['genres']}) - 预测: {score:.4f}")

## 10. 保存模型

In [ ]:
# 保存训练好的模型
recommender.save(
    model_path='./wide_deep_model.h5',
    processor_path='./processor.pkl'
)

print("模型保存成功！")

## 11. 模型评估

评估推荐系统的性能指标

In [ ]:
# 随机选择100个用户进行评估
sample_users = np.random.choice(ratings['user_id'].unique(), size=min(100, len(ratings['user_id'].unique())), replace=False)

print(f"对 {len(sample_users)} 个用户进行推荐质量评估...\n")

recommendation_stats = []

for user_id in sample_users:
    try:
        rec_movies, rec_scores = recommender.recommend(user_id, top_k=10)
        
        if len(rec_movies) > 0:
            recommendation_stats.append({
                'user_id': user_id,
                'num_recommendations': len(rec_movies),
                'avg_score': np.mean(rec_scores),
                'max_score': np.max(rec_scores),
                'min_score': np.min(rec_scores)
            })
    except Exception as e:
        print(f"用户 {user_id} 推荐失败: {e}")

stats_df = pd.DataFrame(recommendation_stats)

print("推荐统计:")
print(stats_df.describe())

# 可视化
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(stats_df['avg_score'], bins=20, edgecolor='black', alpha=0.7)
plt.title('推荐平均得分分布')
plt.xlabel('平均预测得分')
plt.ylabel('用户数')

plt.subplot(1, 2, 2)
plt.boxplot([stats_df['min_score'], stats_df['avg_score'], stats_df['max_score']], 
            labels=['最低分', '平均分', '最高分'])
plt.title('推荐得分箱线图')
plt.ylabel('预测得分')

plt.tight_layout()
plt.show()

## 12. 总结

### Wide & Deep 推荐系统关键特性：

1. **Wide 组件**：
   - 通过交叉特征记忆用户-电影的直接交互
   - 提供可解释性和快速响应

2. **Deep 组件**：
   - 嵌入层将稀疏ID转换为稠密表示
   - 多层神经网络学习复杂的非线性关系
   - 提供良好的泛化能力

3. **多路召回**：
   - 热门召回：保证基础覆盖
   - 类型召回：满足用户偏好
   - 协同过滤：发现潜在兴趣

4. **精排重排**：
   - 使用训练好的模型对候选进行精确打分
   - 平衡准确性和多样性

### 优化建议：
- 增加训练轮数（epochs）以获得更好的性能
- 调整网络层数和单元数
- 添加更多特征（如时间特征、用户画像等）
- 实现更复杂的召回策略
- 添加多样性和新颖性约束